In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from getpass import getpass
from time import sleep

In [13]:
def get_tweet_data(card):
    username = card.find_element(By.XPATH ,".//span").text
    handle = card.find_element(By.XPATH ,".//span[contains(text(),'@')]").text
    try :
        postdate = card.find_element(By.XPATH ,"//article[@data-testid='tweet']/div[1]/div[1]/div[1]/div[2]//time").get_attribute('datetime')
    except NoSuchElementException: 
        pass
    comment = card.find_element(By.XPATH, "./div[2]/div[2]/div[1]").text
    responding = card.find_element(By.XPATH, "./div[2]/div[2]/div[2]").text
    text = comment + responding
    reply_count = card.find_element(By.XPATH, ".//div[@data-testid='reply']").text
    retweet_count = card.find_element(By.XPATH, ".//div[@data-testid='retweet']").text
    like_count = card.find_element(By.XPATH, ".//div[@data-testid='like']").text
    tweet = (username , handle , text , reply_count , retweet_count , like_count)
    return tweet 
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.twitter.com/login")
sleep(10)
username = driver.find_element(By.XPATH, "//input[@name='text']")
username.send_keys('Umairrr_iqbal')
next_button = driver.find_element(By.XPATH, "//span[contains(text(),'Next')]")
next_button.click()
my_password = getpass()
sleep(3)
password = driver.find_element(By.XPATH, "//input[@name='password']")
password.send_keys(my_password)
next_button = driver.find_element(By.XPATH, "//span[contains(text(),'Log in')]")
next_button.click()
sleep(3)
search =  driver.find_element(By.XPATH,"//a[@aria-label='Search and explore']").click()
sleep(3)
enter_keys = driver.find_element(By.XPATH,"//input[@aria-label='Search query']")

enter_keys.send_keys('#cricket')
enter_keys.send_keys(Keys.RETURN)
sleep(1)
latest_button = driver.find_element(By.XPATH, "//span[contains(text(),'Latest')]")
latest_button.click()
tweet_ids = set()
data=[]
last_position = driver.execute_script('return window.pageYOffset;')
scrolling = True
while scrolling:
    sleep(3)
    page_cards = driver.find_elements(By.XPATH ,"//article[@data-testid='tweet']/div[1]/div[1]/div[1]/div[2]" )
    for card in page_cards[-15:] :
        tweet = get_tweet_data(card)
        if tweet :
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script('return window.pageYOffset;')
        if last_position == curr_position:
            scroll_attempt +=1
            if scroll_attempt >=3 :
                scrolling = False 
                break
            else :
                sleep (2)      # attempt to scroll again 
        else : 
            last_position = curr_position 
            break

<ipython-input-13-6844d65d6494>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


········


In [17]:
print(len(data))

134


In [30]:
import csv

In [35]:
with open('cricket_tweets.csv','w',newline = '', encoding='utf-8') as f :
    header = ['Username', 'Handle' , 'Timestamp', 'Comments' , 'Likes ','Rewtweets', 'Text']
    writer = csv.writer(f,delimiter=',')
    writer.writerow(header)
    for i in data:
        writer.writerow(i)